# 02: Missing Values

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
# Load the data
data = pd.read_csv('../datasets/melb_data.csv')

# Set the target column
y = data.Price

# use only numerical features for now 
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# divide data into train/valid
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)